In [1]:
#USE HI-ML-UP-TO-DATE VENV 
import torch
from torch.utils.data import Dataset, DataLoader
from PIL import Image
from pathlib import Path

import os
os.environ['HF_HOME'] = '/vol/biomedic3/bglocker/ugproj2324/nns20/CheXagent/.tuned_projection_model' ## THIS HAS TO BE BEFORE YOU IMPORT TRANSFORMERScd
from transformers import AutoModelForCausalLM, AutoProcessor, GenerationConfig, AdamW, get_cosine_schedule_with_warmup
from torchvision import transforms # THIS HAS TO BE AFTER THE ABOVE LINES


In [2]:

def setup_model() -> tuple:
    device = "cuda"
    dtype = torch.bfloat16

    processor = AutoProcessor.from_pretrained("StanfordAIMI/CheXagent-8b", trust_remote_code=True)
    model = AutoModelForCausalLM.from_pretrained(
        "StanfordAIMI/CheXagent-8b", torch_dtype=dtype, trust_remote_code=True
    ).to(device)
    generation_config = GenerationConfig.from_pretrained("StanfordAIMI/CheXagent-8b")

    return processor, model, device, dtype, generation_config

def generate(images, prompt, processor, model, device, dtype, generation_config):
    inputs = processor(
        images=images, text=f" USER: <s>{prompt} ASSISTANT: <s>", return_tensors="pt"
    ).to(device=device, dtype=dtype)
    output = model.generate(**inputs, generation_config=generation_config)[0]
    response = processor.tokenizer.decode(output, skip_special_tokens=True)
    return response


In [3]:
processor, model, device, dtype, generation_config = setup_model()
        

Loading checkpoint shards:   0%|          | 0/7 [00:00<?, ?it/s]

In [4]:
print(type(model))

<class 'transformers_modules.StanfordAIMI.CheXagent-8b.4934e91451945c8218c267aae9c34929a7677829.modeling_chexagent.CheXagentForConditionalGeneration'>


In [6]:
prompt = 'Describe the findings in the image.'
path = "/vol/biodata/data/chest_xray/mimic-cxr-jpg/files/p10/p10002428/s59891001/df59ff50-56f1fd83-f3664b8b-8e2c67dd-d5f0d039.jpg"
test_png_dset_path = '/vol/biodata/data/chest_xray/VinDr-CXR/1.0.0_png_512/raw/test/688ecdb1a4e994d42b5a50a8c4a9736f.png'
paths = [path]#, test_png_dset_path]
image = Image.open(test_png_dset_path).convert("RGB")
transform = transforms.Compose([transforms.PILToTensor()])
images = [transform(Image.open(path).convert("RGB")) for path in paths]
response = generate(images, prompt, processor, model, device, dtype, generation_config)
print(response)

/vol/biomedic3/bglocker/ugproj2324/nns20/CheXagent/.tuned_projection_model/modules/transformers_modules/StanfordAIMI/CheXagent-8b/4934e91451945c8218c267aae9c34929a7677829/processing_chexagent.py:86: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  [torch.tensor(pixel_values) for pixel_values in encoding_image_processor["pixel_values"]]
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


tmp.shape=torch.Size([2, 1025, 1408])
image_embeds.shape=torch.Size([1, 2, 1025, 1408])
query_output.shape=torch.Size([1, 128, 768])
The cardiomediastinal silhouette is within normal limits. There are bilateral pleural effusions with associated bibasilar atelectasis. There is no pneumothorax.
